<a href="https://colab.research.google.com/drive/178BBrkj8NG51ruo_HndtX8CmzCqJVRt6?usp=sharing"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>

If we want to save data to Google drive, first we should do this:

In [1]:
import os
from google.colab import drive

In [2]:
drive.mount('/content/drive')
os.chdir('drive/My Drive/ChukchiLM')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Download data and baseline implementation from github:

In [ ]:
!git clone https://github.com/ftyers/global-classroom.git

fatal: destination path 'global-classroom' already exists and is not an empty directory.


# Baseline

In [3]:
import os
os.listdir()

['global-classroom',
 'lstm_morph_model',
 'lstm_morph_model_freq_restricted',
 'cnn_embed_morph_model',
 'output1.tsv',
 'dev_output1.tsv',
 'cnn_token_morph_model',
 'output_test_lstm.tsv',
 'output_dev_lstm.tsv',
 'lstm_restricted_test_output.tsv',
 'lstm_restricted_dev_output.tsv']

In [3]:
os.chdir('global-classroom/chukchi/baseline')
!python3 train.py ../data/train.tsv model.dat
!python3 predict.py model.dat < ../data/dev.tsv > output.tsv
!python3 ../evaluate.py ../data/dev.tsv output.tsv 

Written 33724 unigrams and 107835 bigrams to model.dat.
Hits: 33 ; Tokens: 4504
Characters: 37897
Tokens: 8788
Clicks: 37754
Clicks/Token: 4.2960855712335
Clicks/Character: 0.9962266142438716


In [ ]:
!sed 5q < ../data/dev.tsv output.tsv 

ӄԓявыԓя риӄукэтэ ивнин ытри ынкы варкыт гынин ӈэвъэн гэԓгыԓин мэмыԓя рагтыгъэ	ӄ ԓ я в ы ԓ я _ р и ӄ у к э т э _ и в н и н _ ы т р и _ ы н к ы _ в а р к ы т _ г ы н и н _ ӈ э в ъ э н _ г э ԓ г ы ԓ и н _ м э м ы ԓ я _ р а г т ы г ъ э _
ӄԓявыԓ ынкъам купрэн ынанъомрычьын	ӄ ԓ я в ы ԓ _ ы н к ъ а м _ к у п р э н _ ы н а н ъ о м р ы ч ь ы н _
рытэнмавнэн ынӄо эргатык эквэтгъэт копрантыватысӄэквъат	р ы т э н м а в н э н _ ы н ӄ о _ э р г а т ы к _ э к в э т г ъ э т _ к о п р а н т ы в а т ы с ӄ э к в ъ а т _
риӄукэтэ гамгаваны нэнайӈоткоӄэн ӄоԓ	р и ӄ у к э т э _ г а м г а в а н ы _ н э н а й ӈ о т к о ӄ э н _ ӄ о ԓ _
ԓьунин ынӄо йыӈотконэн	ԓ ь у н и н _ ы н ӄ о _ й ы ӈ о т к о н э н _


In output.tsv we should have the original sentence and model output separated by tab. If the model's next word guess was right we append the whole word to output. If the model's guess was wrong, we append to the output each character separately. Between words we should add an underscore

# Our model

In [4]:
os.chdir('..')
os.chdir('..')
os.chdir('..')

In [28]:
import os
import re
import csv

import torch as tt
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from math import ceil
from tqdm import tqdm_notebook

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.nn.utils import clip_grad_norm_

from collections import Counter

import numpy as np

In [6]:
os.listdir("global-classroom/chukchi/data")

['dev.tsv', 'test', 'train.tsv']

In [7]:
with open("global-classroom/chukchi/data/train.tsv") as inp:
  word_sentences = []
  morph_sentences = []
  for line in inp.readlines():
    words, morphs = line.strip('\n').split('\t')
    word_sentences.append(words.split(' '))
    morph_sentences.append(morphs.replace('>',' >').split(' '))

with open("global-classroom/chukchi/data/dev.tsv") as inp:
  dev_word_sentences = []
  dev_morph_sentences = []
  for line in inp.readlines():
    words, morphs = line.strip('\n').split('\t')
    dev_word_sentences.append(words.split(' '))
    dev_morph_sentences.append(morphs.replace('>',' >').split(' '))

with open("global-classroom/chukchi/data/test/test.tsv") as inp:
  test_word_sentences = []
  test_morph_sentences = []
  for line in inp.readlines():
    words, morphs = line.strip('\n').split('\t')
    test_word_sentences.append(words.split(' '))
    test_morph_sentences.append(morphs.replace('>',' >').split(' '))

In [ ]:
print(word_sentences[:3])

[['амаравкэваратэн', 'таа’койӈын'], ['йъйыӄык', 'ныӄэԓпэратӄэн', 'вытэчгытрыӄэргыԓьын', 'йыӈэттэт'], ['мыкыӈ', 'нывытрэтӄин', 'чеԓгатвытрыԓьо', 'ынӄорыым', 'вытэчгытрыԓьо']]


In [ ]:
print(morph_sentences[:3])

[['а', '>маравкэва', '>ра', '>тэн', 'таа', '>’ко', '>йӈы', '>н'], ['йъйыӄы', '>к', 'ны', '>ӄэԓпэр', '>ат', '>ӄэн', 'вытэч', '>гытры', '>ӄэргы', '>ԓьы', '>н', 'йыӈэт', '>тэ', '>т'], ['мык', '>ы', '>ӈ', 'ны', '>вытрэт', '>ӄин', 'чеԓг', '>ат', '>вытры', '>ԓь', '>о', 'ынӄор', '>ыым', 'вытэч', '>гытры', '>ԓь', '>о']]


A hyperparameter: length of input sequence

In [8]:
SEQ_LEN = 5

In [9]:
class MyModel(nn.Module):
    @staticmethod
    def calculate_conv_dim(cnn_input_size,
                           kernel_size, dilation,
                           stride, padding):
      return int(np.floor(((cnn_input_size + 2*padding - dilation *(kernel_size-1)-1)/stride+1)))
    
    def __init__(self, vocab_size, embed_size, kernel_size, dilation,
                 stride, padding, n_kernels):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        self.cnn = nn.Conv1d(in_channels=embed_size, out_channels=n_kernels,
                             kernel_size=kernel_size, dilation=dilation,
                             stride=stride, padding=padding)
        
        self.fc = nn.Linear(self.calculate_conv_dim(SEQ_LEN, kernel_size, dilation, stride, padding) * n_kernels, vocab_size)
        
        self.init_weights()
        
    def init_weights(self):
        nn.init.uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.zeros_(self.fc.bias)
        
    def forward(self, x):
        
        batch_size = x.size(1)
        total_length = x.size(0)
        
        x = self.embedding(x).transpose(2,1)
            
        x = self.cnn(x)
        x = x.reshape(x.shape[0], x.shape[1]*x.shape[2])
        
        #x, _ = nn.utils.rnn.pad_packed_sequence(x, total_length=total_length, batch_first=True)
        
        #x = x.contiguous().view(batch_size * total_length, -1)
        #print(x.shape, self.fc.weight.shape)
        x = self.fc(x)
        #x = x.contiguous().view(total_length, batch_size, -1)
        return x#.transpose(1,2)

Now, let's build a vocab

In [10]:
vocab = Counter([morph for sent in morph_sentences for morph in sent])

In [11]:
len(vocab), sum([v for k,v in vocab.items()])

(19487, 318530)

In [12]:
MIN_COUNT = 1

In [13]:
id2word = [i for i in vocab if vocab[i]>=MIN_COUNT] + ['<bos>','<eos>','<pad>','<unk>']
word2id = {v:k for k,v in enumerate(id2word)}

In [ ]:
word2id['<bos>'], word2id['<eos>'], word2id['<pad>']

(19487, 19488, 19489)

Now let's see the ditribution of sentence length in the training set:

In [ ]:
sent_lens = [len(sent) for sent in morph_sentences]

In [ ]:
pd.Series(sent_lens).describe()

count    30000.000000
mean        10.617667
std          7.877840
min          1.000000
25%          5.000000
50%          9.000000
75%         14.000000
max         94.000000
dtype: float64

In [14]:
def _train_epoch(model, iterator, optimizer, criterion, curr_epoch, device, clip,
                 pad_id):

    model.train()
    
    epoch_loss, accuracy, f1score = 0, 0, 0

    n_batches = len(iterator)
    iterator = tqdm_notebook(iterator, total=n_batches, desc='epoch %d' % (curr_epoch), leave=True)

    for i, (batch_in, batch_out) in enumerate(iterator):
        optimizer.zero_grad()
        
        x = batch_in.to(device)
        pred = model(x)
        
        y = batch_out.to(device)

        loss = criterion(pred, y)

        loss.backward()

        clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        curr_loss = loss.data.cpu().detach().item()

        y_flat, ypred_flat = batch_out.flatten().numpy(), pred.detach().cpu().argmax(dim=1).numpy()

        curr_acc = accuracy_score(y_flat, ypred_flat)
        curr_f1 = f1_score(y_flat, ypred_flat, average='micro')

        accuracy += curr_acc
        f1score += curr_f1

        iterator.set_postfix(loss='%.5f' % curr_loss, acc='%.5f' % curr_acc, f1='%.5f'%curr_f1)

    return epoch_loss/n_batches, accuracy/n_batches, f1score/n_batches

def _test_epoch(model, iterator, criterion, device, pad_id):
    model.eval()
    epoch_loss = 0

    epoch_loss, accuracy, f1score = 0, 0, 0

    n_batches = len(iterator)
    with tt.no_grad():
        for batch_in, batch_out in iterator:
            
            x, y = batch_in.to(device), batch_out.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            epoch_loss += loss.data.cpu().detach().item()

            y_flat, ypred_flat = batch_out.flatten().numpy(), pred.detach().cpu().argmax(dim=1).numpy()

            curr_acc = accuracy_score(y_flat, ypred_flat)
            curr_f1 = f1_score(y_flat, ypred_flat, average='micro')

            accuracy += curr_acc
            f1score += curr_f1

    return epoch_loss / n_batches, accuracy/n_batches, f1score/n_batches


def nn_train(model, train_iterator, valid_iterator, criterion, optimizer, device, n_epochs=100,
          scheduler=None, early_stopping=0, clip=1.0, pad_id=word2id['<pad>']):

    prev_loss = float('inf')

    es_epochs = 0
    best_epoch = None
    history = pd.DataFrame()

    for epoch in range(n_epochs):
        train_loss, train_acc, train_f1 = _train_epoch(model, train_iterator, optimizer, criterion, epoch, device, clip, pad_id)

        print(f"Epoch {epoch} Training loss: {np.round(train_loss, 4)} Training accuracy: {np.round(train_acc, 4)} Training F1: {np.round(train_f1, 4)}")

        valid_loss, valid_acc, valid_f1 = _test_epoch(model, valid_iterator, criterion, device, pad_id)

        print(f"Epoch {epoch} Validation loss: {np.round(valid_loss, 4)} Validation accuracy: {np.round(valid_acc, 4)} Validation F1: {np.round(valid_f1, 4)}")

        if valid_loss < prev_loss:
          print("New record! Saving model")
          tt.save(model, 'cnn_morph_model')

        record = {'epoch': epoch, 'train_loss': train_loss, 'train_acc': train_acc, 'train_f1': train_f1,
                  'valid_loss': valid_loss, 'valid_acc': valid_acc, 'valid_f1': valid_f1}
        history = history.append(record, ignore_index=True)

        if early_stopping > 0:
            if valid_loss > prev_loss:
                es_epochs += 1
            else:
                es_epochs = 0

            if es_epochs >= early_stopping:
                best_epoch = history[history.valid_loss == history.valid_loss.min()].iloc[0]
                print('Early stopping! best epoch: %d val %.5f' % (best_epoch['epoch'], best_epoch['valid_loss']))
                break
                
        prev_loss = min(prev_loss, valid_loss)
    return history

In [15]:
def map_word(word, mapping):
  if word in mapping:
    return mapping[word]
  else:
    return mapping['<unk>']

def make_lm_dataset(sentences, word2id, seq_len):
  text_new = []
  for sent in sentences:
    text_new += ['<pad>' for i in range(seq_len)]
    new_sent = ['<bos>'] + sent
    text_new += new_sent
  
  sents_in = []
  sents_out = []

  for word_id in range(len(text_new)-seq_len):
    sents_in.append(text_new[word_id:word_id+seq_len])
    sents_out.append(text_new[word_id+seq_len])
  return [[map_word(word, word2id) for word in sent] for sent in sents_in], [map_word(word, word2id) for word in sents_out]

In [16]:
def pad_sentences(sents, pad_id, max_len, append_eos=False):
  for sent_id, sent in enumerate(sents):
    if len(sent) < max_len:
      sents[sent_id] = sent + [pad_id for i in range(max_len-len(sent))]
    elif len(sent) < max_len:
      sents[sent_id] = sent[:max_len]
  return sents


def make_batch(batch_in, batch_out, pad_id, max_len):
   max_len = max([len(sent) for sent in batch_in])
   batch_in = pad_sentences(batch_in, pad_id, max_len)
   #batch_out = pad_sentences(batch_out, pad_id, max_len)
   return tt.LongTensor(batch_in), tt.LongTensor(batch_out)

class MyBatchIterator:
  def __init__(self, sents_in, sents_out, batch_size=128,
              pad_id=word2id['<pad>'], max_len=5):
    self.sents_in = sents_in
    self.sents_out = sents_out
    self.batch_size = batch_size
    self.pad_id = pad_id
    self.max_len = max_len
  
  def __iter__(self):
    self.start = 0
    return self
  
  def __next__(self):
    if self.start >= len(self.sents_in):
      raise StopIteration
    batch_in = self.sents_in[self.start:self.start+self.batch_size]
    batch_out = self.sents_out[self.start:self.start+self.batch_size]
    self.start += self.batch_size
    return make_batch(batch_in, batch_out, self.pad_id, self.max_len)
  
  def __len__(self):
    return ceil(len(self.sents_in)/self.batch_size)

Let's prepare our dataset:

In [17]:
sents_in, sents_out = make_lm_dataset(morph_sentences, word2id, SEQ_LEN)
sents_in_dev, sents_out_dev = make_lm_dataset(dev_morph_sentences, word2id, SEQ_LEN)
sents_in_test, sents_out_test = make_lm_dataset(test_morph_sentences, word2id, SEQ_LEN)

X_train, y_train = sents_in, sents_out
X_dev, y_dev = sents_in_dev, sents_out_dev

In [ ]:
sents_in[:10]

[[19489, 19489, 19489, 19489, 19489],
 [19489, 19489, 19489, 19489, 19487],
 [19489, 19489, 19489, 19487, 0],
 [19489, 19489, 19487, 0, 1],
 [19489, 19487, 0, 1, 2],
 [19487, 0, 1, 2, 3],
 [0, 1, 2, 3, 4],
 [1, 2, 3, 4, 5],
 [2, 3, 4, 5, 6],
 [3, 4, 5, 6, 7]]

In [ ]:
sents_out[:10]

[19487, 0, 1, 2, 3, 4, 5, 6, 7, 19489]

In [ ]:
train_iter = MyBatchIterator(X_train, y_train, max_len=SEQ_LEN)
dev_iter = MyBatchIterator(X_dev, y_dev, max_len=SEQ_LEN)

In [ ]:
device = tt.device('cuda')

In [ ]:
model = MyModel(len(word2id), embed_size=100, kernel_size=3, dilation=1, padding=2, stride=1, n_kernels=50).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=word2id["<pad>"]).to(device)
optimizer = tt.optim.Adam(model.parameters())
scheduler = tt.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)

In [ ]:
nn_train(model, train_iter, dev_iter, criterion, optimizer, device, n_epochs=10, scheduler=scheduler)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Epoch 0 Training loss: 0.0 Training accuracy: 0.1641 Training F1: 0.1641
Epoch 0 Validation loss: 4.7532 Validation accuracy: 0.2184 Validation F1: 0.2184
New record! Saving model



Epoch 1 Training loss: 0.0 Training accuracy: 0.2243 Training F1: 0.2243
Epoch 1 Validation loss: 4.7572 Validation accuracy: 0.2337 Validation F1: 0.2337



Epoch 2 Training loss: 0.0 Training accuracy: 0.2622 Training F1: 0.2622
Epoch 2 Validation loss: 4.8963 Validation accuracy: 0.2361 Validation F1: 0.2361



Epoch 3 Training loss: 0.0 Training accuracy: 0.3026 Training F1: 0.3026
Epoch 3 Validation loss: 5.1633 Validation accuracy: 0.2378 Validation F1: 0.2378


...


Epoch 6 Training loss: 0.0 Training accuracy: 0.3792 Training F1: 0.3792
Epoch 6 Validation loss: 5.8676 Validation accuracy: 0.2395 Validation F1: 0.2395



Epoch 7 Training loss: 0.0 Training accuracy: 0.3975 Training F1: 0.3975
Epoch 7 Validation loss: 6.0489 Validation accuracy: 0.2358 Validation F1: 0.2358



Epoch 8 Training loss: 0.0 Training accuracy: 0.4133 Training F1: 0.4133
Epoch 8 Validation loss: 6.1858 Validation accuracy: 0.2327 Validation F1: 0.2327



Epoch 9 Training loss: 0.0 Training accuracy: 0.4273 Training F1: 0.4273
Epoch 9 Validation loss: 6.3701 Validation accuracy: 0.2316 Validation F1: 0.2316


,epoch,train_acc,train_f1,train_loss,valid_acc,valid_f1,valid_loss
0,0.0,0.164129,0.164129,0.0,0.218356,0.218356,4.753221
1,1.0,0.224268,0.224268,0.0,0.233738,0.233738,4.757175
2,2.0,0.262184,0.262184,0.0,0.236104,0.236104,4.896266
3,3.0,0.302600,0.302600,0.0,0.237795,0.237795,5.163336
4,4.0,0.334089,0.334089,0.0,0.238844,0.238844,5.522908
5,5.0,0.358519,0.358519,0.0,0.239485,0.239485,5.689826
6,6.0,0.379195,0.379195,0.0,0.239451,0.239451,5.867639
7,7.0,0.397492,0.397492,0.0,0.235778,0.235778,6.048936
8,8.0,0.413271,0.413271,0.0,0.232688,0.232688,6.185795
9,9.0,0.427298,0.427298,0.0,0.231639,0.231639,6.370078


Npw let's write code for text prediction

In [18]:
def relu(x):
  if x > 0:
    return x
  else:
    return 0

In [19]:
class LMPredictor:
  def __init__(self, model, id2word, word2id):
    self.model = model
    self.id2word = id2word
    self.word2id = word2id

  def predict(self, sent):
    sent = [self.word2id['<pad>']] * relu(SEQ_LEN - len(sent) - 1) + [self.word2id['<bos>']] + [self.word2id[word] if word in self.word2id else self.word2id['<unk>'] for word in sent]
    sent = [sent[-SEQ_LEN:]]
    sent = tt.LongTensor(sent)
    output = model(sent)
    new_word = output.argmax(dim=1)[0]#[-1]
    return self.id2word[new_word]

In [20]:
cpu = tt.device('cpu')
model = tt.load('cnn_morph_model')
model = model.to(cpu)

In [21]:
predictor = LMPredictor(model, id2word, word2id)

In [22]:
sent = dev_morph_sentences[0]
for i in range(len(sent)):
  feed = sent[:i]
  predicted = predictor.predict(feed)
  print(f"Item: {sent[i]} Predicted: {predicted}")

Item: ӄԓявыԓ Predicted: ны
Item: >я Predicted: >ӄэй
Item: риӄукэ Predicted: >ӄэй
Item: >тэ Predicted: >тэ
Item: ив Predicted: >ни
Item: >ни Predicted: >ни
Item: >н Predicted: >н
Item: ытри Predicted: ам
Item: ын Predicted: ӈин
Item: >кы Predicted: >кы
Item: варкы Predicted: >ри
Item: >т Predicted: >т
Item: гын Predicted: ӄынвэр
Item: >ин Predicted: >ин
Item: ӈэвъэ Predicted: ӄыт
Item: >н Predicted: >н
Item: гэ Predicted: ынкъам
Item: >ԓгы Predicted: >ӈэвъэн
Item: >ԓин Predicted: >ԓин
Item: мэмыԓ Predicted: ынкъам
Item: >я Predicted: >ӄэй
Item: ра Predicted: ам
Item: >гты Predicted: >гты
Item: >гъэ Predicted: >гъэ


### Scoring on test set

Let's count word accuracy on test set:

In [23]:
## count accuracy
acc = 0
output = []
dev_test = []
for sent in tqdm_notebook(test_morph_sentences, total=len(test_morph_sentences)):
  sent_acc = 0
  for i in range(len(sent)):
    feed = sent[:i]
    predicted = predictor.predict(feed)
    if predicted == sent[i]:
 #     if predicted == '<eos>':
 #       continue
 #     sent_acc += 1
 #     else:
      output.append(predicted)
      sent_acc += 1
    else:
      output += [c for c in sent[i]]
    if i!=0:
      if i == len(sent)-1:
        output.append('_')
      elif not sent[i+1].startswith('>'):
        output.append('_')
  output.append('\n')
  acc += sent_acc/len(sent)
acc = acc/len(test_morph_sentences)
listToStr = ' '.join(map(str, output)).replace('>',' ')
listToStr = re.sub(' +', ' ', listToStr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [24]:
output1 = list(listToStr.split('\n'))

In [25]:
output1[:10]

['г ы м нин _ ы т л ь а т _ ы ’ т т ъ у в и _ ы н к ъ а м _ о ’ м р ы т в а а л _ ',
 ' п у у р ъ у э п ы _ г ы м нин _ о ’ м _ о ’ м р ы к в о т _ н э м ы ӄ э й _ ӄ о р а г ы н р э т ы л ь о _ н и т ӄин _ ',
 ' г ы м нин _ ы т л ы г ы т _ ы т л ь ’ а т э _ ӄ о н п ы _ э м н у ӈ к ы _ ны м и г ч и р эт ӄ и н э т _ н э м ы ӄ э й _ ӄ о р а г ы н р э т ы л ь о _ н и т ӄ и н э т _ ',
 ' ы н к ъ а м м у р и _ г а ч а к э т т о м г а _ г э _ г э е г т э л ь м у р и _ к а в р а г ы р г ы н _ ы н к ъ а м _ г ы м _ ',
 ' ы т р ъ э ч е _ ӈ и р э ӄ _ н э н э н э т _ в а г ъ э т _ ',
 ' ы т р ъ э ч ',
 ' к ы т у р ы н ӈ ы т э ӄ э _ м э д в э д э в _ п р э з и д э н т о _ э н м а _ к а н ч а л я н э т ы _ г э р э м к и ч и л ь и н _ ',
 ' ы н к ъ а л ю у т _ ӈ ы р ъ а _ в э р т а л ё т т э _ в а к ъ о г ъ а т ӈ а _ ',
 ' ны н ы п ч е ӈ и в й и в ӄ и н _ к о л ё _ н ы м н ы м а _ м э д в э д э в ъ ы м _ ',
 ' г э ч е в кы _ ны н т ы ӄин _ т а ӈ к о л ё _ ы н кы _ ны г ы н р и т ӄин _ ']

In [26]:
!sed 10q < global-classroom/chukchi/data/test/test.tsv

гымнин ытльат ы’ттъуви ынкъам о’мрытваал	гым>нин ытльа>т ы’ттъуви ынкъам о’мрытваал>
пууръу эпы гымнин о’м о’мрыквот нэмыӄэй ӄорагынрэтыльо нитӄин	пууръу эпы гым>нин о’м о’мрыквот нэмыӄэй ӄора>гынрэты>ль>о н>ит>ӄин
гымнин ытлыгыт ытль’атэ ӄонпы эмнуӈкы нымигчирэтӄинэт нэмыӄэй ӄорагынрэтыльо нитӄинэт	гым>нин ытлыгы>т ытль’а>т>э ӄонпы эмнуӈ>кы ны>мигчир>эт>ӄинэ>т нэмыӄэй ӄора>гынрэты>ль>о нитӄинэт
ынкъам мури гачакэттомга гэ гэегтэльмури каврагыргын ынкъам гым	ынкъам мури га>чакэт>томг>а гэ гэ>егтэль>мури каврагыргы>н ынкъам гым
ытръэче ӈирэӄ нэнэнэт вагъэт	ытръэч>е ӈирэӄ нэнэнэ>т ва>гъэ>т
ытръэч	ытръэч
кытур ынӈытэӄэ мэдвэдэв прэзидэнто энма канчалянэты гэрэмкичильин	кытур ынӈытэӄ>э мэдвэдэв прэзидэнт>о эн>ма канчалян>эты гэ>рэмкичи>льин
ынкъа люут ӈыръа вэрталёттэ вакъогъатӈа	ынкъа люут ӈыръа вэрталёт>тэ вакъо>гъа>т>ӈа
ныныпчеӈивйивӄин колё нымныма мэдвэдэвъым	ны>ны>пчеӈи>в>йив>ӄин колё нымным>а мэдвэдэв>>ъым
гэчевкы нынтыӄин таӈколё ынкы ныгынритӄин	гэчев>кы ны>нты>ӄин таӈ>колё ын>кы 

In [29]:
with open("global-classroom/chukchi/data/test/test.tsv") as inp:
  for line in inp.readlines():
    words, morphs = line.strip('\n').split('\t')
    dev_test.append(words)
  dev_test.append('\n')

with open('cnn_token_test_output.tsv', 'w') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerows(zip(dev_test, output1))

In [30]:
acc

0.12700522523119506

In [31]:
!sed 10q < cnn_token_test_output.tsv

гымнин ытльат ы’ттъуви ынкъам о’мрытваал	г ы м нин _ ы т л ь а т _ ы ’ т т ъ у в и _ ы н к ъ а м _ о ’ м р ы т в а а л _ 
пууръу эпы гымнин о’м о’мрыквот нэмыӄэй ӄорагынрэтыльо нитӄин	 п у у р ъ у э п ы _ г ы м нин _ о ’ м _ о ’ м р ы к в о т _ н э м ы ӄ э й _ ӄ о р а г ы н р э т ы л ь о _ н и т ӄин _ 
гымнин ытлыгыт ытль’атэ ӄонпы эмнуӈкы нымигчирэтӄинэт нэмыӄэй ӄорагынрэтыльо нитӄинэт	 г ы м нин _ ы т л ы г ы т _ ы т л ь ’ а т э _ ӄ о н п ы _ э м н у ӈ к ы _ ны м и г ч и р эт ӄ и н э т _ н э м ы ӄ э й _ ӄ о р а г ы н р э т ы л ь о _ н и т ӄ и н э т _ 
ынкъам мури гачакэттомга гэ гэегтэльмури каврагыргын ынкъам гым	 ы н к ъ а м м у р и _ г а ч а к э т т о м г а _ г э _ г э е г т э л ь м у р и _ к а в р а г ы р г ы н _ ы н к ъ а м _ г ы м _ 
ытръэче ӈирэӄ нэнэнэт вагъэт	 ы т р ъ э ч е _ ӈ и р э ӄ _ н э н э н э т _ в а г ъ э т _ 
ытръэч	 ы т р ъ э ч 
кытур ынӈытэӄэ мэдвэдэв прэзидэнто энма канчалянэты гэрэмкичильин	 к ы т у р ы н ӈ ы т э ӄ э _ м э д в э д э в _ п р э з и д э н т о _ э н

In [32]:
!sed 10q < global-classroom/chukchi/baseline/output_test.tsv

гымнин ытльат ы’ттъуви ынкъам о’мрытваал	г ы м н и н _ ы т л ь а т _ ы ’ т т ъ у в и _ ынкъам _ о ’ м р ы т в а а л _
пууръу эпы гымнин о’м о’мрыквот нэмыӄэй ӄорагынрэтыльо нитӄин	п у у р ъ у _ э п ы _ г ы м н и н _ о ’ м _ о ’ м р ы к в о т _ н э м ы ӄ э й _ ӄ о р а г ы н р э т ы л ь о _ н и т ӄ и н _
гымнин ытлыгыт ытль’атэ ӄонпы эмнуӈкы нымигчирэтӄинэт нэмыӄэй ӄорагынрэтыльо нитӄинэт	г ы м н и н _ ы т л ы г ы т _ ы т л ь ’ а т э _ ӄ о н п ы _ э м н у ӈ к ы _ н ы м и г ч и р э т ӄ и н э т _ н э м ы ӄ э й _ ӄ о р а г ы н р э т ы л ь о _ н и т ӄ и н э т _
ынкъам мури гачакэттомга гэ гэегтэльмури каврагыргын ынкъам гым	ынкъам _ м у р и _ г а ч а к э т т о м г а _ г э _ г э е г т э л ь м у р и _ к а в р а г ы р г ы н _ ынкъам _ г ы м _
ытръэче ӈирэӄ нэнэнэт вагъэт	ы т р ъ э ч е _ ӈ и р э ӄ _ н э н э н э т _ в а г ъ э т _
ытръэч	ы т р ъ э ч _
кытур ынӈытэӄэ мэдвэдэв прэзидэнто энма канчалянэты гэрэмкичильин	к ы т у р _ ы н ӈ ы т э ӄ э _ м э д в э д э в _ п р э з и д э н т о _ э н м а _ к 

In [33]:
os.listdir()

['global-classroom',
 'lstm_morph_model',
 'lstm_morph_model_freq_restricted',
 'cnn_embed_morph_model',
 'output1.tsv',
 'dev_output1.tsv',
 'cnn_morph_model',
 'output_test_lstm.tsv',
 'output_dev_lstm.tsv',
 'lstm_restricted_test_output.tsv',
 'lstm_restricted_dev_output.tsv',
 'cnn_token_test_output.tsv']

In [34]:
!python3 global-classroom/chukchi/evaluate.py global-classroom/chukchi/data/test/test.tsv cnn_token_test_output.tsv 

Characters: 37927
Tokens: 8374
Clicks: 37334
Clicks/Token: 4.458323381896346
Clicks/Character: 0.9843647006090648


### Scoring on dev set

In [35]:
## count accuracy
dev_acc = 0
output = []
dev_test = []
for sent in tqdm_notebook(dev_morph_sentences, total=len(dev_morph_sentences)):
  sent_acc = 0
  for i in range(len(sent)):
    feed = sent[:i]
    predicted = predictor.predict(feed)
    if predicted == sent[i]:
      output.append(predicted)
      sent_acc += 1
    else:
      output += [c for c in sent[i]]
    if i!=0:
      if i == len(sent)-1:
        output.append('_')
      elif not sent[i+1].startswith('>'):
        output.append('_')
  output.append('\n')
  dev_acc += sent_acc/len(sent)
dev_acc = dev_acc/len(dev_morph_sentences)
listToStr = ' '.join(map(str, output)).replace('>',' ')
listToStr = re.sub(' +', ' ', listToStr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [36]:
output1 = list(listToStr.split('\n'))

In [37]:
dev_acc

0.23686578967710895

In [38]:
## write output.tsv

with open("global-classroom/chukchi/data/dev.tsv") as inp:
  for line in inp.readlines():
    words, morphs = line.strip('\n').split('\t')
    dev_test.append(words)
  dev_test.append('\n')
import csv
with open('cnn_token_dev_output.tsv', 'w') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerows(zip(dev_test, output1))

In [39]:
!sed 10q < cnn_token_dev_output.tsv

ӄԓявыԓя риӄукэтэ ивнин ытри ынкы варкыт гынин ӈэвъэн гэԓгыԓин мэмыԓя рагтыгъэ	ӄ ԓ я в ы ԓ я _ р и ӄ у к э тэ _ и в ни н _ ы т р и _ ы н кы _ в а р к ы т _ г ы н ин _ ӈ э в ъ э н _ г э ԓ г ы ԓин _ м э м ы ԓ я _ р а гты гъэ _ 
ӄԓявыԓ ынкъам купрэн ынанъомрычьын	 ӄ ԓ я в ы ԓ ы н к ъ а м _ к у п р э н _ ы н а н ъ о м р ы ч ь ы н _ 
рытэнмавнэн ынӄо эргатык эквэтгъэт копрантыватысӄэквъат	 р ы т э н м а в н э н _ ынӄо _ э р г атык _ э к в э т г ъ э т _ к о п р а н т ы в а т ы с ӄ э к в ъ а т _ 
риӄукэтэ гамгаваны нэнайӈоткоӄэн ӄоԓ	 р и ӄ у к э тэ _ г а м г а в а н ы _ н э н а й ӈ о т к о ӄэн _ ӄ о ԓ _ 
ԓьунин ынӄо йыӈотконэн	 ԓ ь у ни н _ ы н ӄ о _ й ы ӈ о т к о н э н _ 
 эвын коԓё	 э в ы н _ к о ԓ ё _ 
тыкгаткэта ивнин риӄукэтэ	 т ы к г а т к э т а _ и в ни н _ р и ӄ у к э тэ _ 
йыӈотконэн эвын коԓё	 й ы ӈ о т к о н э н _ э в ы н _ к о ԓ ё _ 
тыкгаткэта	 т ы к г а т к э т а _ 
ивнин риӄукэтэ ынкы	 и в ни н _ р и ӄ у к э тэ _ ы н кы _ 


In [40]:
!python3 global-classroom/chukchi/evaluate.py global-classroom/chukchi/data/dev.tsv cnn_token_dev_output.tsv 

Characters: 37897
Tokens: 8788
Clicks: 36252
Clicks/Token: 4.125170687300865
Clicks/Character: 0.9565928701480328
